# history

In [3]:
import os,glob
import json
import pickle
import copy
import math
import numpy as np
import pandas as pd
import seaborn as sns

###from sklearn.cluster import MeanShift
###from sklearn.cluster import KMeans
###from sklearn.metrics import pairwise_distances
from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
import SimpleITK as sitk

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm

#from matplotlib.patches import Patch
# import hdbscan
# import umap

from swc_handler import parse_swc

# import graphviz
# from graphviz import Digraph

import networkx as nx
from networkx.drawing.nx_pydot import pydot_layout
from networkx.drawing.nx_pydot import graphviz_layout
from networkx.drawing.nx_pydot import write_dot

# 1 function to arrange module

In [2]:
# def get_regions_name_module_dict(ms=[]):
#     with open('../../assets/modules.txt','r') as f:
#         data = []
#         for i,line in enumerate(f.readlines()):
#             if (ms) and (i not in ms): continue
                
#             line = line.strip()
#             print(line)
#             for region_name in line.split(', '):
#                 data.append((region_name,i))
    
#     new_dict = dict(data)
#     return new_dict

def get_regions_name_module_dict():
    data = []
    with open('../../assets/modules.txt','r') as f:
        i = 0
        for line in f.readlines():
            line = line.strip()
            region_name = line.split('\'')[1]
            data.append((region_name,i))
            if line[-1]==';': 
                i += 1
    new_dict = dict(data)
    return new_dict



from utils import color_map
regions_module_dict = color_map(get_regions_name_module_dict(),
                                colors=['orange','magenta','cyan'])
                                #colors=['chocolate','orange','yellow','black','blue','cyan','purple','red','magenta']+['white']*6)
                                #colors=['white']*9+['orange','yellow','black','blue','purple','magenta'])
print(regions_module_dict)  

{0: 'orange', 1: 'magenta', 2: 'cyan'}
{'PC5': 'orange', 'MARN': 'orange', 'DMX': 'orange', 'PRP': 'orange', 'AMB': 'orange', 'PPY': 'orange', 'RPA': 'orange', 'EW': 'orange', 'Pa4': 'orange', 'IV': 'orange', 'MA3': 'orange', 'CLI': 'orange', 'DR': 'orange', 'RPO': 'orange', 'V': 'orange', 'VTN': 'orange', 'AT': 'orange', 'LT': 'orange', 'SLC': 'orange', 'CS': 'orange', 'GPi': 'magenta', 'VTA': 'magenta', 'PPN': 'magenta', 'PSTN': 'cyan', 'RT': 'cyan', 'ZI': 'cyan', 'PeF': 'cyan', 'TU': 'cyan', 'SPA': 'cyan', 'ME': 'cyan', 'PN': 'cyan', 'III': 'cyan', 'MRN': 'cyan'}


# 2 function to cluster all regions

In [34]:
from utils import get_dict, get_u16_u32_id_dict, get_region_name_list
def mykmeans(X,n_clusters):  
    np.set_printoptions(suppress=True)
    X = np.array(X).round(2)
    X_internal = X[1:]-X[:-1]
    ini_clusters_X_internal_ids_list = sorted(range(len(X_internal)),key=lambda x: X_internal[x],reverse=True)[:n_clusters]

    cluster = {}
    for ini_cluster_X_internal_id in ini_clusters_X_internal_ids_list:
        cluster[ini_cluster_X_internal_id] = [X[ini_cluster_X_internal_id]]      
    data = X.tolist()
    for k,v in cluster.items():
        for d in v:
            data.pop(data.index(d))
    
    while len(data):
        data_dist_tomax = []
        data_dist_tomin = []
        for c in cluster.keys():
            [data_dist_tomax.append(abs(d-max(cluster[c]))) for d in data]
            [data_dist_tomin.append(abs(d-min(cluster[c]))) for d in data]
        min_maxdist = min(data_dist_tomax)
        min_mindist = min(data_dist_tomin)
        rm_id_dist = data_dist_tomax.index(min_maxdist) if min_maxdist<min_mindist else data_dist_tomin.index(min_mindist)
        rm_id = rm_id_dist%len(data)
        rm_data = data[rm_id]
        rm_c = int(rm_id_dist/len(data))
        dc = list(cluster.keys())[rm_c]
        cluster[dc].extend([rm_data])                
        data.pop(data.index(rm_data))

    yhat = []
    for x in X:
        xc = 0
        for c,d in cluster.items():
            if x in d:
                xc = 1
                yhat.append(c)
                break
        if not xc: raise
    return yhat

def get_region_range_and_region_cluster(level,uint,axis,n_clusters):
    center_dict = get_dict(level,uint,'center')
    if not (axis=='DV' and uint==32):
        u16_u32_id_dict = get_u16_u32_id_dict()
        
    raxis = ['AP','DV','RL'].index(axis)
    if not (axis=='DV' and uint==32):
        rc_list = [(k,v[raxis]) for k,v in center_dict.items() if not np.isnan(v[raxis])]
    else:
        center_dict2 = get_dict(level,16,'center')
        rc_list = [(u16_u32_id_dict[k],v[raxis]) for k,v in center_dict2.items() if (int(k)%2 and (not np.isnan(v[raxis])))]
    rc_list.sort(key=lambda x: x[1],reverse=False)
    region_list = [r[0] for r in rc_list]
    center_list = [r[1] for r in rc_list]
    
    axis2 = ['RL','AP','DV'][['AP','DV','RL'].index(axis)]
    raxis2 = ['AP','DV','RL'].index(axis2)
    if not (axis=='DV' and uint==32):
        rc_list2 = [(k,v[raxis2]) for k,v in center_dict.items() if not np.isnan(v[raxis2])]
    else:
        rc_list2 = [(u16_u32_id_dict[k],v[raxis2]) for k,v in center_dict2.items() if (int(k)%2 and (not np.isnan(v[raxis2])))]
    rc_list2.sort(key=lambda x: x[1],reverse=False)
    region_list2 = [r[0] for r in rc_list2]
    center_list2 = [r[1] for r in rc_list2]
   
    yhat = mykmeans(center_list,n_clusters)
    cs,ccs = np.unique(yhat,return_counts=True)
    count_max = ccs.max()
    print(f'\nn{n_clusters} len{len(center_list)} clustering......{yhat} len{len(yhat)}......n{len(np.unique(yhat))}......{count_max}')
    while count_max > n_clusters:
        n_clusters += 2 
        yhat = mykmeans(center_list,n_clusters=n_clusters)
        cs,ccs = np.unique(yhat,return_counts=True)
        count_max = ccs.max()
        print(f'\nn{n_clusters} len{len(center_list)} clustering......{yhat} len{len(yhat)}......n{len(np.unique(yhat))}......{count_max}')

    outfig_file = f'../plots/{axis}{level}u{uint}regions_n{n_clusters}cluster.png'
    plot_axis_cluster(axis,yhat,region_list,center_list,outfig_file)
    
    clusters_list = []
    for ilabel,label in enumerate(np.unique(yhat)):
        cluster = np.array(region_list)[np.array(yhat)==label].tolist()
        clusters_list.append([r for r in region_list2 if r in cluster])

    return region_list,region_list2,clusters_list

def plot_axis_cluster(axis,yhat,region_list,center_list,outfig_file):    
    n_clusters = len(np.unique(yhat))
    cs = 'brgcmyk'
    k = int(np.ceil(n_clusters / len(cs)))
    color = [c for c in (cs * k)[:n_clusters]]
    fig = plt.figure(figsize=(math.ceil(n_clusters/10),1)) 
    xlims = [(0,528),(0,320),(0,456)][['AP','DV','RL'].index(axis)]
    plt.xlim(xlims)
    label_list = []
    for i,label in enumerate(yhat):
        point = center_list[i]
        if not label in label_list: 
            if len(label_list): 
                plt.scatter(x=np.array(tmp), y=0 * np.array(tmp), s=2, c=color[len(label_list)])  
            label_list.append(label)
            tmp = [point]
        else:
            tmp.append(point)        
    plt.savefig(outfig_file,dpi=600)
    plt.close()
    
    
level = 316
uint = 32
axis = 'AP'
n_clusters = 40
regions_list, regions_list2, regions_cluster = get_region_range_and_region_cluster(level,uint,axis,n_clusters)
# regions_name_list = get_region_name_list(regions_list,uint)
# regions_name_cluster = [get_region_name_list(cluster,uint) for cluster in regions_cluster]
for i in ['GPi','VTA','PPN']:#从上到下 从左到右
    print(i,get_region_name_list(regions_list,32).index(i),get_region_name_list(regions_list2,32).index(i),)


n40 len314 clustering......[0, 1, 2, 5, 5, 5, 6, 7, 8, 10, 10, 12, 12, 13, 16, 16, 16, 18, 18, 21, 21, 21, 27, 27, 27, 27, 27, 27, 34, 34, 34, 34, 34, 34, 34, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 54, 54, 54, 54, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 115, 115, 115, 115, 115, 115, 115, 124, 124, 124, 124, 124, 124, 124, 124, 124, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 192, 192, 192, 192, 192, 192, 192, 193, 208, 208, 208, 208, 208, 208, 208, 208, 208, 208, 208, 208, 208, 208, 208, 220, 220, 

# 3 plot graphviz

In [36]:
from utils import get_n1327_n4_u32dict
regions_structure_dict = color_map(get_n1327_n4_u32dict(),
                                   colors=['white','yellow','blue','red','green'])
                                   #colors=['white','pink','gold','skyblue','green'])
print(get_region_name_list([512,623,688,343],32))

level = 316
uint = 32
regions_neighbor_dict = get_dict(level,uint,key='neighbor')   

G = nx.DiGraph()
for i,region_list in enumerate(regions_cluster[:-1]):
    regions_name_list = get_region_name_list(region_list,uint)
    if not i: 
        for n,r in zip(regions_name_list,region_list): 
            color = regions_structure_dict[r]
            if color=='white': print(f'\n{n} not in structure');continue
            if n in regions_module_dict.keys():
                fillcolor = regions_module_dict[n]
                penwidth = 10
            else: 
                fillcolor = 'white'
                penwidth = 10
            G.add_node(n,height=2,width=3.5,fontsize=80,style='filled',
                       color=color,fillcolor=fillcolor,penwidth=penwidth)
            
    next_cluster = regions_cluster[i+1]
    next_cluster_regions_name = get_region_name_list(next_cluster,uint)
    for n,r in zip(next_cluster_regions_name,next_cluster):
        color = regions_structure_dict[r]
        if color=='white': continue
        if n in regions_module_dict.keys():
            fillcolor = regions_module_dict[n]
            penwidth = 10
        else: 
            fillcolor = 'white'
            penwidth = 10
        G.add_node(n,height=2,width=3.5,fontsize=80,style='filled',
                   color=color,fillcolor=fillcolor,penwidth=penwidth)
    print(i,regions_name_list,next_cluster_regions_name)
    
    for region,region_name in zip(region_list,regions_name_list):
        if region_name not in G.nodes: continue
        for n,r in zip(next_cluster_regions_name,next_cluster):
            if n not in G.nodes: continue
            if r in regions_neighbor_dict[region]:
                G.add_edge(region_name, n, color='black', penwidth=10, arrowsize=0)
            else:
                G.add_edge(region_name, n, color='white', penwidth=0, arrowsize=0)

TR = nx.nx_agraph.to_agraph(G)
TR.draw("../plots/graph.pdf", prog='dot')

#TR.draw("../plots/graph15.pdf", prog='dot')

# TR.graph['label'] = f'\n{cur_label}'
# nodes = TR.nodes
# nodes[rname]['penwidth'] = pw
# nodes[rname]['fillcolor'] = color
# nodes[rname]['style'] = 'filled'

{0: 'white', 512: 'yellow', 623: 'blue', 688: 'red', 343: 'green'}
['CB', 'CNU', 'CTX', 'BS']
0 ['MOB'] ['AOB']
1 ['AOB'] ['FRP']
2 ['FRP'] ['AON', 'ORBm', 'ORBvl']
3 ['AON', 'ORBm', 'ORBvl'] ['ORBl']
4 ['ORBl'] ['PL']
5 ['PL'] ['TT']
6 ['TT'] ['AId', 'ILA']
7 ['AId', 'ILA'] ['MOs', 'DP']
8 ['MOs', 'DP'] ['AIv']
9 ['AIv'] ['SH', 'MOp', 'ACB']
10 ['SH', 'MOp', 'ACB'] ['ACAd', 'OT']
11 ['ACAd', 'OT'] ['CLA', 'GU', 'SSp-m']
12 ['CLA', 'GU', 'SSp-m'] ['IG', 'MS', 'ACAv', 'NDB', 'LSr', 'LSv']
13 ['IG', 'MS', 'ACAv', 'NDB', 'LSr', 'LSv'] ['ADP', 'OV', 'PS', 'SI', 'AVPV', 'AVP']
14 ['ADP', 'OV', 'PS', 'SI', 'AVPV', 'AVP'] ['MEPO']
15 ['MEPO'] ['FS', 'MPO', 'VMPO', 'PVpo', 'LSc', 'MA', 'BST', 'CP', 'TRS', 'MPN', 'LPO', 'SSp-ul', 'VLPO', 'SF', 'BAC', 'PD']
16 ['FS', 'MPO', 'VMPO', 'PVpo', 'LSc', 'MA', 'BST', 'CP', 'TRS', 'MPN', 'LPO', 'SSp-ul', 'VLPO', 'SF', 'BAC', 'PD'] ['EPd', 'SSp-n', 'PIR', 'AAA']
17 ['EPd', 'SSp-n', 'PIR', 'AAA'] ['PVa', 'ASO', 'AD', 'SSs', 'RCH', 'PT', 'SSp-ll', 'AM', 'NL

# 4 plot corr graphviz 

In [52]:
from utils import load_json
print('\n',len(regions_module_dict.keys()),list(regions_module_dict.keys()))
for region in ['GPi','VTA','PPN']:
    corr_regions = load_json(f'../../assets/{region}.json')
    print('\n',region,'\nhigh corr: ',corr_regions)
    
    new_regions_cluster = []
    no_module = []
    high_module = []
    for cluster in regions_cluster:
        cluster_name = get_region_name_list(cluster,32)
        if len(set(cluster_name)&set(corr_regions)):
            new_cluster = []
            for ic,c in enumerate(cluster):
                c_name = cluster_name[ic]
                if c_name in corr_regions:
                    new_cluster.append(c)
                    if not c_name in list(regions_module_dict.keys()): no_module.append(c_name)
                    else:high_module.append(c_name)                   
            new_regions_cluster.append(new_cluster)
    print(f'not in module: {no_module}')
    print(f'high corr and in module: {high_module}')
    
    G = nx.DiGraph()
    for i,region_list in enumerate(new_regions_cluster[:-1]):
        regions_name_list = get_region_name_list(region_list,uint)
        if not i:  
            for n,r in zip(regions_name_list,region_list): 
                color = regions_structure_dict[r]
                if color=='white': print(f'\n{n} not in structure');continue
                if n in regions_module_dict.keys():
                    fillcolor = regions_module_dict[n]
                    penwidth = 10
                else: 
                    fillcolor = 'white'
                    penwidth = 10
                G.add_node(n,height=2,width=3.5,fontsize=80,style='filled',
                           color=color,fillcolor=fillcolor,penwidth=penwidth)
                print(f'{i} node {n}')
                    
        next_cluster = new_regions_cluster[i+1]
        next_cluster_regions_name = get_region_name_list(next_cluster,uint)             
        for n,r in zip(next_cluster_regions_name,next_cluster):
            color = regions_structure_dict[r]
            if color=='white': continue
            if n in regions_module_dict.keys():
                fillcolor = regions_module_dict[n]
                penwidth = 10
            else: 
                fillcolor = 'white'
                penwidth = 10
            G.add_node(n,height=2,width=3.5,fontsize=80,style='filled',
                       color=color,fillcolor=fillcolor,penwidth=penwidth)
            print(f'{i+1} node {n}')

        for r,region_name in zip(region_list,regions_name_list):
            if region_name not in G.nodes: continue
            for n,r in zip(next_cluster_regions_name,next_cluster):
                if n not in G.nodes: continue
                if r in regions_neighbor_dict[r]:
                    G.add_edge(region_name, n, color='black', penwidth=10, arrowsize=0)
                    print(f'edge [{region_name}, {n}]')
                else:
                    G.add_edge(region_name, n, color='white', penwidth=0, arrowsize=0)

    TR = nx.nx_agraph.to_agraph(G)
    TR.draw(f"../plots/{region}.pdf", prog='dot')   


 33 ['PC5', 'MARN', 'DMX', 'PRP', 'AMB', 'PPY', 'RPA', 'EW', 'Pa4', 'IV', 'MA3', 'CLI', 'DR', 'RPO', 'V', 'VTN', 'AT', 'LT', 'SLC', 'CS', 'GPi', 'VTA', 'PPN', 'PSTN', 'RT', 'ZI', 'PeF', 'TU', 'SPA', 'ME', 'PN', 'III', 'MRN']

 GPi 
high corr:  ['AId', 'AIp', 'AIv', 'MRN', 'LPO', 'LSv', 'MA', 'BST', 'CEA', 'TU', 'VISC', 'VM', 'VPLpc', 'DG', 'VPMpc', 'VTA', 'OT', 'ZI', 'EPv', 'FS', 'GPi', 'PPN', 'GU', 'IA', 'ME', 'PN']
not in module: ['AId', 'AIv', 'OT', 'GU', 'LSv', 'FS', 'MA', 'BST', 'LPO', 'AIp', 'IA', 'VISC', 'CEA', 'EPv', 'VM', 'VPMpc', 'VPLpc', 'DG']
high corr and in module: ['GPi', 'TU', 'ZI', 'ME', 'VTA', 'PN', 'MRN', 'PPN']
0 node AId
1 node AIv
2 node OT
3 node GU
4 node LSv
5 node FS
5 node MA
5 node BST
5 node LPO
6 node AIp
7 node IA
7 node VISC
7 node GPi
7 node CEA
7 node TU
7 node EPv
7 node VM
8 node ZI
8 node ME
8 node VPMpc
9 node VPLpc
10 node VTA
10 node DG
11 node PN
12 node MRN
13 node PPN

 VTA 
high corr:  ['III', 'ISN', 'AIp', 'MRN', 'LPO', 'RPO', 'RT', 'MA', '

# history

# function to manage data to axis

In [ ]:
def get_comp_acronym_data(df_labels,Id,Acronym,allow0,axis_id_list_dict,axis_acronym_list_dict):      
    Id_tree = get_tree_from('u16_id')
    Id_number = [Id_tree[v][Id] for v in df_labels['n671_u16_id']]  
    Id_number = [Id_tree[v][Id] for v in df_labels['n671_u16_id']]  
    #print(len(Id_number),len(np.unique(Id_number)))
    Id_count = df_labels['count'].values 
    tree = get_tree_from(Id)
    Id_list =  list(tree.keys())
    #print(len(Id_list),len(np.unique(Id_list)))
    Id_list_count = [Id_count[np.array(Id_number)==i].sum() for i in Id_list]       
    
    sorted_Id_count_index = sorted(range(len(Id_list_count)), key = lambda x:Id_list_count[x], reverse=True)
    sorted_Id_count = [Id_list_count[i] for i in sorted_Id_count_index]
    sorted_Id_list = [Id_list[i] for i in sorted_Id_count_index]    

    #axis range
    axis_data_dict = {}
    for axis in ['AP','DV','RL']:  
        axis_id_list = axis_id_list_dict[axis]
        new_data = []

        #brain range
        brain_todel = []
        for brain in df_labels['brain']: 
            if brain in brain_todel: continue
            brain_todel.append(brain)

            df_brain = df_labels[df_labels['brain']==brain]
            if len(df_brain):
                brain_Id_list = [Id_tree[v][Id] for v in df_brain['n671_u16_id']]   
                brain_Id_count = df_brain['count'].values 
                if not allow0:
                    for i in axis_id_list:
                        comp = sorted_Id_list.index(i)
                        acronym = Id_tree[i][Acronym]
                        brain_i_count = brain_Id_count[np.array(brain_Id_list)==i].sum()   
                        new_data.append([comp,acronym,i,brain_i_count,brain,df_brain['label'].values[0]])        

        new_df_labels = pd.DataFrame(np.array(new_data),columns=['comp','acronym',Id,'count','brain','label'])#acronym顺序是axis顺序
        axis_data_dict[axis] = new_df_labels
    return axis_data_dict

# plot labelled brain regional info

In [ ]:
# def plot_labellingbrains_feature(df_labels,
#                                  allow0,
#                                  level,
#                                  hemi,
#                                  comp,
#                                  outfig_dir,
#                                  outfig_end,
#                                ):
    
#     Level = f'in_n{level}'
#     Id = 'u16_id' if hemi else 'u32_id'
#     Acronym = f'acronym_lr' if hemi else f'acronym'
#     Center = 'center_u25_lr' if hemi else 'center_u25'
#     axis_id_list_dict, axis_acronym_list_dict = get_acronym_list(Level,Id,Acronym,Center)
    
#     Id = f'n{level}_u16_id' if hemi else f'n{level}_u32_id'
#     Acronym = f'acronym_lr' if hemi else f'acronym'
#     axis_comp_data_dict = get_comp_acronym_data(df_labels,Id,Acronym,allow0,axis_id_list_dict,axis_acronym_list_dict)
    
#     for axis in ['AP','DV','RL']: 
#         print(axis)  
#         print(axis_id_list_dict[axis])
#         print(axis_acronym_list_dict[axis])
#         axis_data = axis_comp_data_dict[axis]
#         axis_data_count = axis_data['count'].values.astype(int)
#         axis_data['count'] = np.ceil(axis_data_count/axis_data_count.max()*20)/20
#         axis_comp_data = axis_data[axis_data['comp'].apply(int)<=comp]
#         print(axis_comp_data,'\n')
    
#         cs = 'brgcmyk'
#         nlabel = len(np.unique(axis_comp_data['label']))
#         k = int(np.ceil(nlabel / len(cs)))
#         palette = [c for c in (cs * k)[:nlabel]]
        
#         g = sns.relplot(
#             data=axis_comp_data,
#             x='acronym',
#             y='brain',
#             size='count',
#             hue='label',
#             height=20,
#             aspect=1,
#             sizes=(0,100),
#             size_norm=(0,1),
#             palette=palette,
#         )        
#         plt.xticks(fontsize=5,rotation=90)#axis_comp_data['acronym'].values
#         plt.xlabel('')
#         plt.yticks(labels=None, fontsize=5)
#         plt.ylabel('Brain', fontsize=20)
#         plt.grid(alpha=0.5)        
#         outfig_compfile = f'{outfig_dir}/{axis}_n{level}_hemi{hemi}_comp{comp}_{outfig_end}'
#         plt.savefig(outfig_compfile,dpi=300)
#         plt.close()
        
#         print('\n\n')
#     return axis_comp_data
  
    
    
# allow0 = False       
# level = 316
# hemi = True
# comp = 200
# outfig_dir = '../plots/labelling'
# test = plot_labellingbrains_feature(df_labels = df_labels_signal,
#                              allow0 = allow0,
#                              level = level,
#                              hemi = hemi,
#                              comp = comp,
#                              outfig_dir = outfig_dir,
#                              outfig_end = 'signal.png'
#                             )

In [ ]:
# allow0 = False       
# level = 70
# hemi = True
# comp = 200
# outfig_dir = '../plots/labelling'
# plot_labellingbrains_feature(df_labels = df_labels_signal_new,
#                              allow0 = allow0,
#                              level = level,
#                              hemi = hemi,
#                              comp = comp,
#                              outfig_dir = outfig_dir,
#                              outfig_end = 'signal_new.png'
#                             )

# function to cluster regions along axis

In [ ]:
# [t for t in get_new_tree() if np.isnan(t['center_25um_lr']).sum()]

# function to rerange data to certain order

In [ ]:
def rerange(df,acronym_list,key='count'):
    plot = []
    for a,acronym in enumerate(acronym_list):
        df_acronym = df[df['acronym']==acronym]
        if a==0: brain = df_acronym['brain'].values
        count = df_acronym[key].values.tolist()
        plot.append(count)
    #print(f'rerange acronym num: {len(plot)}, brain num: {len(plot[0])}')
    plot = np.array(plot).T
    df_plot = pd.DataFrame(plot,columns=acronym_list,index=brain,)
    return df_plot


def row_mean(df_brains,label_brain_dict):
    labels_cols = df_brains.columns
    labels_data = []
    labels_index = []
    for i,lbs in enumerate(label_brain_dict.items()):
        l,bs = lbs
        labels_index.append(l)
        label_data = df_brains[[(int(b) in bs) for b in df_brains.index]].mean(axis=0).tolist()
        labels_data.append(label_data)

    df_labels = pd.DataFrame(labels_data,index=labels_index,columns=labels_cols)
    return df_labels

# labels regional signal cluster 

In [ ]:
# def plot_labelling_correlation_clustermap(df_labels,label_brain_dict,
#                                           level,hemi,Cluster,
#                                           outfig_dir,outfig_end):
#     Level = f'in_n{level}'
#     Id = 'u16_id' if hemi else 'u32_id'
#     Acronym = f'acronym_lr' if hemi else f'acronym'
#     Center = 'center_u25_lr' if hemi else 'center_u25'
#     axis_id_list_dict, axis_acronym_list_dict = get_acronym_list(Level,Id,Acronym,Center)

#     Id = f'n{level}_u16_id' if hemi else f'n{level}_u32_id'
#     Acronym = f'acronym_lr' if hemi else f'acronym'
#     axis_comp_data_dict = get_comp_acronym_data(df_labels,Id,Acronym,allow0,axis_id_list_dict,axis_acronym_list_dict)
    
#     for axis in ['AP']:#,'DV','RL']: 
#         print(axis)  
#         print(axis_id_list_dict[axis])
#         print(axis_acronym_list_dict[axis])
#         axis_data = axis_comp_data_dict[axis]
#         axis_data['count'] = axis_data['count'].values.astype(int)
#         print(axis_data,'\n')
    
#         axis_acronym_list = axis_acronym_list_dict[axis]
#         acronym_count = rerange(axis_data,axis_acronym_list)
#         print(acronym_count,'\n')
        
#         label_acronym_count = row_mean(acronym_count,label_brain_dict)
#         label_acronym_count = pd.DataFrame(label_acronym_count.values.T,columns=label_acronym_count.index,index=label_acronym_count.columns)
#         print(label_acronym_count,'\n')
       
#         label_corr = label_acronym_count.corr()
#         print(label_corr,'\n')
        
# #         for i,label in enumerate(label_list):
# #             print('\n',i,label)
# #             if label=='all': df_label = df_labels
# #             else: df_label = df_labels[df_labels['label']==label]
            
#         palette = sns.color_palette("Greens",n_colors=len(label_brain_dict))     
            
#         g = sns.clustermap(label_corr
#         #            , norm=LogNorm(),
#         #            , zscore=0#1
#         #            , standard_scale=0#1       
#         #            , mask=mask       #only show these true
#                     , figsize=(20,20)
#                     , cmap='coolwarm'
#                     , cbar_pos=(0.01,0.05,0.02,0.15)#left bottom width height
#                     , dendrogram_ratio=(.1,.2)
#                     , row_cluster=Cluster
#                     , row_colors=palette
#                     , col_cluster=Cluster
#                     , col_colors=palette
#         #             , vmax=.3
#         #             , vmin=.1
#         #             , center=0.5
#         #             , annot=True
#         #             , xticklabels=True
#         #             , yticklabels=True
#         #             , square=True
#                    )
#         plt.xticks(fontsize=5,rotation=90)#axis_comp_data['acronym'].values
#         plt.xlabel('')
#         plt.yticks(labels=None, fontsize=5)
#         plt.ylabel('')
#         plt.grid(alpha=0.5)
#         outfig_compfile = f'{outfig_dir}/{axis}_n{level}_hemi{hemi}_{Cluster}clusterlabel_{outfig_end}'
#         plt.savefig(outfig_compfile,dpi=300)
#         plt.close()

#         print('\n\n')
            
#         #axis_acronym_list = g.data2d.columns
            
#     return label_corr
             
# level = 316
# hemi = True
# outfile_dir = '../data'
# outfig_dir = '../plots/labelling'
# for cluster in [False,True]:
#     test = plot_labelling_correlation_clustermap(df_labels=df_labels_signal,
#                                      label_brain_dict = labels_brains_dict,
#                                      level=level,
#                                      hemi=hemi,
#                                      Cluster=cluster,
#                                      outfig_dir = outfig_dir,
#                                      outfig_end = 'signal.png'
#                                      )


In [ ]:
# level = 316
# hemi = True
# cluster = False
# outfile_dir = '../data'
# outfig_dir = '../plots/labelling'

# for cluster in [False,True]:
#     test = plot_labelling_correlation_clustermap(df_labels=df_labels_signal_new,
#                                      label_brain_dict = labels_brains_dict_new,
#                                      level=level,
#                                      hemi=hemi,
#                                      Cluster=cluster,
#                                      outfig_dir = outfig_dir,
#                                      outfig_end = 'signal_new.png'
#                                      )

# region corr heatmap of one label 

In [ ]:
# def plot_acronym_correlation_heatmap(df_labels,level,hemi,
#                                      comp,label_list,
#                                      outfig_dir,outfig_end,
#                                      clusters_dict):
#     Level = f'in_n{level}'
#     Id = 'u16_id' if hemi else 'u32_id'
#     Acronym = f'acronym_lr' if hemi else f'acronym'
#     Center = 'center_u25_lr' if hemi else 'center_u25'
#     axis_id_list_dict, axis_acronym_list_dict = get_acronym_list(Level,Id,Acronym,Center)

#     Id = f'n{level}_u16_id' if hemi else f'n{level}_u32_id'
#     Acronym = f'acronym_lr' if hemi else f'acronym'
#     for label in label_list:
#         print(label,'\n')
#         if label=='all': df_label = df_labels
#         else: df_label = df_labels[df_labels['label']==label]        
#         axis_comp_data_dict = get_comp_acronym_data(df_label,Id,Acronym,allow0,axis_id_list_dict,axis_acronym_list_dict)

#         for axis in ['AP','DV','RL']: 
#             print(axis)
#             axis_acronym_list = axis_acronym_list_dict[axis]
            
#             axis_data = axis_comp_data_dict[axis]
#             axis_data['count'] = axis_data['count'].values.astype(int)
#             print(axis_data.shape,'\n')
#             axis_comp_data = axis_data[axis_data['comp'].apply(int)<=comp]
#             print(axis_comp_data.shape,'\n')
        
#             #acronym_count = axis_comp_data.pivot(index=['brain'],columns=['acronym'],values=['count'])
#             acronym_count = rerange(axis_comp_data,axis_acronym_list)
#             print(acronym_count.shape,'\n')       
#             acronym_corr = acronym_count.corr().fillna(0)  
#             print(acronym_corr.shape,'\n')
            
#             g,ax = plt.subplots(figsize=(60,60))
#             sns.heatmap(acronym_corr
#             #            ,mask=mask       #只显示为true的值
#                         , cmap='coolwarm'
#                         , cbar_kws={"shrink": 0.4}
#             #             , vmax=.3
#             #             , vmin=.1
#             #             , center=0.5
#             #             , annot=True
#                         , xticklabels=True
#                         , yticklabels=True
#                         , square=True
#                        )
#             plt.xticks(fontsize=5,rotation=90)#axis_comp_data['acronym'].values
#             plt.xlabel('')
#             plt.yticks(labels=None, fontsize=5)
#             plt.ylabel('')
#             plt.grid(alpha=0.5)
#             outfig_compfile = f'{outfig_dir}/{label}_{axis}_n{level}_hemi{hemi}_comp{comp}_corr_{outfig_end}'
#             plt.savefig(outfig_compfile,dpi=300)
#             plt.close()
                       
#             print('\n\n')            
            


# label_list = []
# for i,l in enumerate(labels_brains_dict.items()):
#     k,v = l
#     if len(v)>5:
#         print(i+1,k,len(v),'\n')
#         label_list.append(k)
# label_list = ['all'] + label_list

# level = 316
# hemi = True
# comp = 632
# outfig_dir = '../plots/corr/n316_c632'
# plot_acronym_correlation_heatmap(df_labels=df_labels_signal,
#                                  level=level,
#                                  hemi=hemi,
#                                  comp=comp,
#                                  label_list=label_list,
#                                  outfig_dir = outfig_dir,
#                                  outfig_end = 'signal.png',
#                                  clusters_dict=clusters_dict,
#                                  )

In [ ]:
# label_list = []
# for i,l in enumerate(labels_brains_dict_new.items()):
#     k,v = l
#     if len(v)>5:
#         print(i+1,k,len(v),'\n')
#         label_list.append(k)
# label_list = ['all'] + label_list

# level = 316
# hemi = True
# comp = 632
# outfig_dir = '../plots/corr/n316_c632_new'
# plot_acronym_correlation_heatmap(df_labels=df_labels_signal_new,
#                                  level=level,
#                                  hemi=hemi,
#                                  comp=comp,
#                                  label_list=label_list,
#                                  outfig_dir = outfig_dir,
#                                  outfig_end = 'signal_new.png',
#                                  clusters_dict=clusters_dict,
#                                  )

# 3 region corr clustermap of one label

In [ ]:
def plot_acronym_correlation_clustermap(df_labels,level,hemi,
                                     label_list,
                                     outfig_dir,outfig_end,
                                     clusters_dict):
    Level = f'in_n{level}'
    Id = 'u16_id' if hemi else 'u32_id'
    Acronym = f'acronym_lr' if hemi else f'acronym'
    Center = 'center_u25_lr' if hemi else 'center_u25'
    axis_id_list_dict, axis_acronym_list_dict = get_acronym_list(Level,Id,Acronym,Center)
    comp = 1000
    
    #cmap
    cmap_dict = {}
    for axis in ['AP','DV','RL']: 
        #print('\n',axis)
        axis_cluster_dict = clusters_dict[axis]
        n_clusters = len(axis_cluster_dict)
        #print(f'{n_clusters} clustering cmap\n\n')
        cs = (sns.color_palette("Greys",n_colors=n_clusters))
        #print(len(cs),cs)
        cmap = []
        for nc,c in enumerate(axis_cluster_dict.items()):
            k,v = c
            for na in range(len(v)):
                cmap.append(cs[nc])
        cmap_dict[axis] = cmap
    
    Id = f'n{level}_u16_id' if hemi else f'n{level}_u32_id'
    Acronym = f'acronym_lr' if hemi else f'acronym'  
    for i,label in enumerate(label_list):
        print('\n',i,label)
        if label=='all': df_label = df_labels
        else: df_label = df_labels[df_labels['label']==label]
            
        axis_comp_data_dict = get_comp_acronym_data(df_label,Id,Acronym,allow0,axis_id_list_dict,axis_acronym_list_dict)    

        for axis in ['AP']:#,'DV','RL']: 
            print('\n',axis)
            axis_data = axis_comp_data_dict[axis]
            axis_comp_data = axis_data[axis_data['comp'].apply(int)<=comp]#comp
            axis_comp_data['count'] = axis_comp_data['count'].values.astype(int)

            if label=='all':
                axis_acronym_list = axis_acronym_list_dict[axis]
                acronym_count = rerange(axis_comp_data,axis_acronym_list)
                Cluster = True
                cmap = cmap_dict[axis]
            else:
                acronym_count = rerange(axis_comp_data,axis_acronym_list)
                Cluster = False
                cmap = cmap 
                print('\n',axis_acronym_list,axis_comp_data.shape,acronym_count.shape)
     
            acronym_corr = acronym_count.corr().fillna(0)  
            #print('\n',acronym_corr)
            
            g = sns.clustermap(acronym_corr
            #            , norm=LogNorm(),
            #            , zscore=0#1
            #            , standard_scale=0#1       
            #            , mask=mask       #only show these true
                        , figsize=(100,100)
                        , cmap='coolwarm'
                        , cbar_pos=(0.01,0.05,0.02,0.15)#left bottom width height
                        , dendrogram_ratio=(.1,.2)
                        , row_cluster=Cluster
                        , row_colors=cmap
                        , col_cluster=Cluster
                        , col_colors=cmap
            #             , vmax=.3
            #             , vmin=.1
            #             , center=0.5
            #             , annot=True
            #             , xticklabels=True
            #             , yticklabels=True
                        , square=True
                       )
            plt.xticks(fontsize=1,rotation=90)#axis_comp_data['acronym'].values
            plt.xlabel('')
            plt.yticks(labels=None, fontsize=1)
            plt.ylabel('')
            plt.grid(alpha=0.5)
            outfig_compfile = f'{outfig_dir}/{label}_{axis}_n{level}_hemi{hemi}_comp{comp}_clustercorr_{outfig_end}'
            plt.savefig(outfig_compfile,dpi=300)
            plt.close()
            
            print('\n\n')
            
            if label=='all': 
                axis_acronym_list = axis_acronym_list.tolist()
                acronym_index_list = [axis_acronym_list.index(a) for a in g.data2d.columns]                
                cmap = [cmap[i] for i in acronym_index_list]
                axis_acronym_list = g.data2d.columns
                
    return axis_acronym_list


# level = 316
# hemi = True
# outfig_dir = '../plots/cluster_corr/n316'
# label_list = []
# for i,l in enumerate(labels_brains_dict.items()):
#     k,v = l
#     if len(v)>5:
#         print(i+1,k,len(v),'\n')
#         label_list.append(k)
# label_list = ['all'] + label_list

# axis_acronym_list = plot_acronym_correlation_clustermap(df_labels=df_labels_signal,
#                                      level=level,
#                                      hemi=hemi,
#                                      label_list=label_list,
#                                      outfig_dir = outfig_dir,
#                                      outfig_end = 'signal.png',
#                                      clusters_dict=clusters_dict,
#                                      )

In [ ]:
level = 316
hemi = False
outfig_dir = '../plots/cluster_corr/n316_false'
label_list = []
for i,l in enumerate(labels_brains_dict_new.items()):
    k,v = l
    if len(v)>5:
        print(i+1,k,len(v),'\n')
        label_list.append(k)
label_list = ['all'] + label_list

#NOTE
label_list = ['all']

axis_acronym_list_new = plot_acronym_correlation_clustermap(df_labels=df_labels_signal_new,
                                     level=level,
                                     hemi=hemi,
                                     label_list=label_list,
                                     outfig_dir = outfig_dir,
                                     outfig_end = 'signal_new.png',
                                     clusters_dict=clusters_dict,
                                     )
axis_acronym_list_new = axis_acronym_list_new.tolist()
print(axis_acronym_list_new)

# projection mapped to row of level315 and hemi

In [ ]:
def plot_projection(projection_file,outfig,axis_acronym_list):
    projection_csv = pd.read_csv(projection_file,index_col=0)
    
    acronym_lr_list = []
    for a in projection_csv.columns:
        acronym_lr = a.split('_',1)[1]+'_l' if a.startswith('ipsi') else a.split('_',1)[1]+'_r'
        acronym_lr_list.append(acronym_lr)
    projection_csv.columns = acronym_lr_list

#     tree = get_tree()
#     n316_u16_list = [t['acronym_lr'] for t in tree if t['in_n316']]
#     rm = [i for i in acronym_lr_list if i not in n316_u16_list]
#     print(rm)
#     for r in rm:
#         projection_csv = projection_csv.drop(projection_csv[projection_csv[r]>0].index)

    #projection_csv = projection_csv.reset_index(drop=True)
    
    projection_csv = (projection_csv.T/projection_csv.sum(axis=1)).T.round(2)
    
    #col reraange
    projection_csv_range = projection_csv[axis_acronym_list] #row rerange: sort_df_grade = df_grade.loc[sort_list]
    g = sns.clustermap(projection_csv_range,row_cluster=True,col_cluster=False,cmap='coolwarm')#,center=-0.1)
    plt.savefig(outfig+'.png')
    plt.close()    
    projection_csv_range = g.data2d
    projection_csv_range.index= list(range(projection_csv_range.values.shape[0]))
    
#     for i in range(28):#74):
#         begin = i*100+0
#         end = i*100+100
#         projection = projection_csv_range.iloc[begin:end]
#         shape =  projection.values.shape
#         projection.index= list(range(shape[0]))
#         print('\n',i,shape)#projection,'\n')
                
#         fig,axes = plt.subplots(shape[0])
#         for n,p in projection.iterrows():
#             #print(n)
#             img = np.array([(p.values)*255]).astype(np.uint8)#NOTE to use cmap
#             img = np.tile(img,(int(shape[1]/shape[0]),1))
#             img3d = np.expand_dims(img,axis=2).repeat(3,axis=2)
#             axes[n].imshow(img,cmap='coolwarm')
#             axes[n].set_xticks([])
#             axes[n].set_yticks([])
#             axes[n].axis('off')
#             axes[n].set_ylabel(str(n))
#         plt.gcf().subplots_adjust(left=None,top=None,bottom=None,right=None)
#         outfig_file = outfig+'_'+str(i)+'.png'
#         plt.savefig(outfig_file,dpi=600) 
#         plt.close()
#         print(outfig_file)
        
    return projection_csv,projection_csv_range,
    
# projection_file = '../data/2743_Axon_Projection.csv'#NOTE should clac yourself later
# outfig = '../plots/cluster_corr/projection.png'
# test = plot_projection(projection_file,outfig,axis_acronym_list)

In [ ]:
projection_file = '../data/2743_Axon_Projection.csv'#NOTE should clac yourself later
outfig = '../plots/cluster_corr/projection_new'
test0,test1 = plot_projection(projection_file,outfig,axis_acronym_list_new)

In [ ]:
neuron_list = test0.index.tolist()
projection_dict = {}
for n,p in test1.iterrows():
    #print(n,p)
    nmax = p.max()
    nindex = p.values.tolist().index(nmax)
    npro = p[nindex-5:nindex+5]
    isproindex = np.nonzero(npro.values>0)[0]
    if len(isproindex)>2:
        nneuron = neuron_list[n]
        print(nneuron,nindex,' -5:+5 ',npro.values.tolist())
        nacronym = npro.index.tolist()
        print(nacronym,'\n')
        projection = str([nacronym[i] for i in isproindex])
        if projection in projection_dict.keys():
            projection_dict[projection].append(nneuron)
        else: 
            projection_dict[projection] = [nneuron]
print('\n\n\n')
for projection,pneuron in projection_dict.items():
    pnum = len(pneuron)
    print(projection,pneuron,pnum,'\n')        

In [ ]:
for projection,pneuron in projection_dict.items():
    if 'AUDp_l' in projection or 'AUDp_r' in projection:
        pnum = len(pneuron)
        print(projection,pneuron,pnum)

In [ ]:
for projection,pneuron in projection_dict.items():
    pnum = len(pneuron)
    if 'POST_l' in projection or 'POST_r' in projection:
            print(projection,pneuron,pnum)

In [ ]:
for projection,pneuron in projection_dict.items():
    pnum = len(pneuron)
    if 'RSPv_l' in projection or 'RSPv_r' in projection:
            print(projection,pneuron,pnum)

In [ ]:
for projection,pneuron in projection_dict.items():
    if 'MOs_l' in projection or 'MOs_r' in projection:
        pnum = len(pneuron)
        print(projection,pneuron,pnum)

In [ ]:
for projection,pneuron in projection_dict.items():
    if 'VM_l' in projection or 'VM_r' in projection:
        pnum = len(pneuron)
        print(projection,pneuron,pnum)

# 4 graphviz of one label without cluster

In [ ]:
def produce_colorcode(n_colors=7,cmap='Blues'):
    cs = sns.color_palette(cmap,n_colors=n_colors)
    new_cs = []
    for c in cs:
        new_c = [int(256*i) for i in c]
        new_c_16 = '#'+''.join([hex(i)[2:].zfill(2) for i in new_c])
        new_cs.append(new_c_16)
    return new_cs

def plot_graph_nx(acronym_plot,
               neighbor_edge,
               module_dict,
               cmap,
               method,
               outdot_graphfile,
               outfig_graphfile,
               ):
    G = nx.DiGraph()
    
    node_list = list(acronym_plot.index)
    nodesize_list = acronym_plot.values.astype('int').tolist()
    nodefillcolor_list = [cmap[c] for c in nodesize_list]
    G.add_nodes_from(node_list) 
    
    edge_list = []
    edgecolor_list = []
    if neighbor_edge:
        pass
    else:
        for i,ai in enumerate(node_list): 
            for aj in node_list[i+1:]:
                edge_list.append((ai,aj))
                edgecolor_list.append(module_dict[ai])
    G.add_edges_from(edge_list) 
    write_dot(G,outdot_graphfile)
    
#     print(len(G.nodes),G.nodes,'\n')
#     print(len(G.edges),G.edges,'\n')
#     write_dot(G,outdot_graphfile)
#     nx.draw(G,
#             with_labels=True,
#             font_size=2,
#             font_color='black',
#             arrowsize=2,
#             pos = graphviz_layout(G,prog=method,),
#             node_size=nodesize_list,
#             node_color=nodefillcolor_list,
#     #         cmap=plt.cm.Blues,
#     #         vmin=round(min(plot_node_size)),
#     #         vmax=round(max(plot_node_size)),
#             edge_color=edgecolor_list,
#     #         width=plot_edge_width,
#            )
#     plt.savefig(outfig_graphfile,dpi=600) 
           
    TR = nx.transitive_reduction(G)   
    TR.add_nodes_from(G.nodes(data=True))
    TR.add_edges_from((u, v, G.edges[u, v]) for u, v in TR.edges)
    print(len(TR.nodes),TR.nodes,'\n')
    print(len(TR.edges),TR.edges,'\n')
    
    edge_list_new = TR.edges
    edgecolor_list_new = [edgecolor_list[edge_list.index(e)] for e in edge_list_new ]
    nx.draw(TR,
            with_labels=True,
            font_size=2,
            font_color='black',
            arrowsize=2,
            pos = graphviz_layout(TR,prog=method,),
            node_size=nodesize_list,
            node_color=nodefillcolor_list,
    #         cmap=plt.cm.Blues,
    #         vmin=round(min(plot_node_size)),
    #         vmax=round(max(plot_node_size)),
            edge_color=edgecolor_list_new,
    #         width=plot_edge_width,
           )
    plt.savefig(outfig_graphfile,dpi=600)    
    
    return G,TR

In [ ]:
# def plot_graph_graphviz(acronym_plot,
#                neighbor_edge,
#                pattern_dict,
#                cmap,
#                method,
#                outdot_graphfile,
#                outfig_graphfile,
#                ):
#     G = Digraph()
#     G.clear()
#     G = Digraph('G', filename='cluster.gv', format='png',
#                 engine=method,
#                 node_attr={'width':'0.2','height':'0.8','fontsize':'20',},
#                 graph_attr={'dpi':'300'},
#                 #'cmap':'Blues',
#                 )
#     #g.graph_attr['dpi'] = '300'

#     G.attr(compound='true',fontsize='10')
    
#     acronym_list = list(acronym_plot.index)
#     for i,acronym in enumerate(acronym_list): 
#         comp = int(acronym_plot[acronym])
#         G.node(name=acronym, 
#                label=acronym,#+'_  size_'+str(size),
#                color=pattern_dict[acronym],
#                style='filled',
#                fillcolor=cmap[comp],
#                       )
    
#     if neighbor_edge:
#         pass
#     else:
#         for i,ai in enumerate(acronym_list): 
#             for aj in acronym_list[i+1:]:
#                 G.edge(ai,aj,
#                        color = pattern_dict[ai],
#                        )
    
#     G.render(outfig_graphfile, view=True)          
#     return G


# G.save('../plots/graph/test.dot')   
# G_ = nx.nx_agraph.read_dot('../plots/graph/test.dot')

# TR = nx.nx_agraph.from_agraph(G)  # convert back to networkx (but as Graph)

In [ ]:
def calc_dist(xyz1,xyz2):
    qsum = 0
    for i in range(len(xyz1)):
        qsum += (xyz1[i]-xyz2[i])**2
    return math.sqrt(qsum)


def plot_acronym_graph(df_labels,level,hemi,plot2,
                       clusters_dict,
                       module_dict,                        
                       outfig_dir,outfig_end):
    Level = f'in_n{level}'
    Id = 'u16_id' if hemi else 'u32_id'
    Acronym = f'acronym_lr' if hemi else f'acronym'
    Center = 'center_u25_lr' if hemi else 'center_u25'
    axis_id_list_dict, axis_acronym_list_dict = get_acronym_list(Level,Id,Acronym,Center)
 
    if level==671: uplevel=316
    if level==316: uplevel=70
    if level==70: uplevel=5
    UPLevelId = f'n{uplevel}_u16_id' if hemi else f'n{uplevel}_u32_id'
    level_dict = {}
    for t in get_new_tree():
        if t[Level]:
            level_dict[t[Acronym]] = t[UPLevelId]
    print(len(level_dict),level_dict)
    uplevel_list = np.unique(list(level_dict.values())).tolist()
    level_num = len(uplevel_list)
    print(level_num,uplevel_list)
    levelmap = produce_colorcode(n_colors=level_num,cmap='Accent')
    print(len(levelmap),levelmap)
    level_map = dict(zip(uplevel_list,levelmap))
    print(len(level_map),level_map)
    
    Id = f'n{level}_u16_id' if hemi else f'n{level}_u32_id'
    Acronym = f'acronym_lr' if hemi else f'acronym'
    axis_comp_data_dict = get_comp_acronym_data(df_labels,Id,Acronym,allow0,axis_id_list_dict,axis_acronym_list_dict)    
        
    for axis in ['AP']:#,'DV','RL']:
        print('\n')
        axis_data = axis_comp_data_dict[axis]
        axis_data = axis_data.astype({'comp': int})
        axis_data = axis_data.astype({'count': int})
        print(axis,'\n',axis_data)
        
        axis_acronym_list = axis_acronym_list_dict[axis]
        acronym_num = len(axis_acronym_list)
        cmap = produce_colorcode(n_colors=acronym_num+1,cmap='Blues')
        
        axis_data = rerange(axis_data,axis_acronym_list,key=plot2)  
        print(plot2,'\n',axis_data)
        if plot2=='count':    
            agg = 'sum'
            ex_str = f'axis_data.{agg}(axis=0)' 
            acronym_plot = eval(ex_str)  
            acronym_plot = (acronym_plot/acronym_plot.max()*acronym_num).astype(int)            
        if plot2=='comp':
            acronym_plot = axis_data.iloc[0]
        print(plot2,'\n',acronym_plot) 

        cluster_dict = clusters_dict[axis]
        axis2 = ['RL','AP','DV'][['AP','DV','RL'].index(axis)]
        #['centerz','centerx','centery'][['AP','DV','RL'].index(axis)]
        axis2_acronym_list = axis_acronym_list_dict[axis2]
        print(axis2,'\n',axis2_acronym_list)
        cluster_dict_new = {}
        for c,ca in cluster_dict.items():
            ca_range = [axis2_acronym_list.tolist().index(a) for a in ca]
            ca_range.sort(reverse=True)
            cluster_dict_new[c] = [axis2_acronym_list[ia] for ia in ca_range]
            print(c,ca,cluster_dict_new[c],)
        print('\n')
        
        method = 'dot'
        neighbor_edge = False   
        outfig_graphfile = f'{outfig_dir}/{plot2}_{axis}_n{level}_hemi{hemi}_edgeneigh{neighbor_edge}_graph_{method}_{outfig_end}'     
        outdot_graphfile = f'{outfig_graphfile}.dot'  

        G = plot_graph(acronym_plot,
                       neighbor_edge,
                       cluster_dict_new,
                       module_dict,
                       cmap,
                       method,
                       outdot_graphfile,outfig_graphfile)

#         G = plot_graph_new(acronym_plot,
#                        neighbor_edge,
#                        cluster_dict_new,
#                        module_dict,
#                        level_dict,
#                        level_map,
#                        method,
#                        outdot_graphfile,outfig_graphfile)
        
        return cluster_dict,axis2_acronym_list,G
#     for a in brains_acronyms_count_df.columns:
#         neighbor_id = acronym_tree[a][neighbor]
#         neighbor_acronym = [id_tree[i][acronym] for i in neighbor_id]
#         plot_tree[a] = {'neighbor':neighbor_acronym,
#                         center:acronym_tree[a][center], 
#                         'count':int(brains_acronyms_count_df[a].mean()),
#                         voxel:acronym_tree[a][voxel], 
#                         'rgb':acronym_tree[a]['rgb'],
#                         'soma':len(np.nonzero((soma_acronym_df[acronym]==a).values)[0]),
#                        } 


def plot_graph(acronym_plot,
                       neighbor_edge,
                       cluster_dict,
                       module_dict,
                       cmap,
                       method,
                       outdot_graphfile,outfig_graphfile):
    node_list = list(acronym_plot.index)
    nodesize_list = acronym_plot.values.astype('int').tolist()
    nodefillcolor_list = [cmap[c] for c in nodesize_list]
    #                fillcolor=cmap[comp],
    
    g = Digraph('G', filename='cluster.gv', format='pdf',
                engine=method,
                node_attr={'width':'0.2','height':'0.8','fontsize':'20',
                           #'cmap':'Blues',
                          },
                #graph_attr={'dpi':'300'},
               )
#     g.graph_attr['dpi'] = '300'
    g.attr(compound='true',fontsize='10')    

    edges_cluster = []
    edge_cluster = []
    for ii,cca, in enumerate(cluster_dict.items()):
        c,ca = cca
        print('\nsubgraph',c,ca)
        with g.subgraph(name=f'cluster_{c}') as s:
            s.attr(label=str(c),fontsize='20',fontcolor='black',style='dashed',
                   #color='grey',rank='same',
                  )
            for node in ca: 
                if node in node_list:
                    ica = node_list.index(node)
                    nodesize = nodesize_list[ica]
                    nodefillcolor = nodefillcolor_list[ica]
#                     fillcolor = np.round(new_maps[sorted(nodes_color_list).index(color)],2) #信号密度
#                     fillcolor = f'0.1,0.1,{fillcolor[2]}'
                    nodecolor = module_dict[node]
                    print(node,nodecolor,nodesize,nodefillcolor)
                    s.node(name=node, 
                           label=node,
                           color=nodecolor,
                           style='filled',
                           fillcolor=nodefillcolor,)
                
        if ii==0: 
            edge_cluster.append(str(c));edge_cluster.append(ca[int(len(ca)/2)])
        else : 
            edge_cluster.append(str(c));edge_cluster.append(ca[int(len(ca)/2)])
            print('cluster edge: ',edge_cluster)
            edges_cluster.append(edge_cluster)
            edge_cluster = [str(c),ca[int(len(ca)/2)]]

        if neighbor_edge:    
            print('neighbor developed')
            
        print('\n')
               
    for i,edge in enumerate(edges_cluster):
        g.edge(edge[1],edge[3],
               lhead='cluster_'+edge[2],
               ltail='cluster_'+edge[0],
               len='2',
               color = 'black',
              )

    #print(g.source)
    g.render(outfig_graphfile,view=True)
    return g


def plot_graph_new(acronym_plot,
                       neighbor_edge,
                       cluster_dict,
                       module_dict,
                       level_dict,
                       cmap,
                       method,
                       outdot_graphfile,outfig_graphfile):
    node_list = list(acronym_plot.index)
    nodesize_list = acronym_plot.values.astype('int').tolist()
    nodefillcolor_list = [cmap[level_dict[c] ]for c in node_list]
    
    g = Digraph('G', filename='cluster.gv', format='pdf',
                engine=method,
                node_attr={'width':'0.2','height':'0.8','fontsize':'20',
                           #'cmap':'Blues',
                          },
                #graph_attr={'dpi':'300'},
               )
#     g.graph_attr['dpi'] = '300'
    g.attr(compound='true',fontsize='10')    

    edges_cluster = []
    edge_cluster = []
    for ii,cca, in enumerate(cluster_dict.items()):
        c,ca = cca
        print('\nsubgraph',c,ca)
        with g.subgraph(name=f'cluster_{c}') as s:
            s.attr(label=str(c),fontsize='20',fontcolor='black',style='dashed',
                   #color='grey',rank='same',
                  )
            for node in ca: 
                if node in node_list:
                    ica = node_list.index(node)
                    nodesize = nodesize_list[ica]
                    nodefillcolor = nodefillcolor_list[ica]
#                     fillcolor = np.round(new_maps[sorted(nodes_color_list).index(color)],2) #信号密度
#                     fillcolor = f'0.1,0.1,{fillcolor[2]}'
                    nodecolor = module_dict[node]
                    print(node,nodecolor,nodesize,nodefillcolor)
                    s.node(name=node, 
                           label=node,
                           color=nodecolor,
                           style='filled',
                           fillcolor=nodefillcolor,)
                
        if ii==0: 
            edge_cluster.append(str(c));edge_cluster.append(ca[int(len(ca)/2)])
        else : 
            edge_cluster.append(str(c));edge_cluster.append(ca[int(len(ca)/2)])
            print('cluster edge: ',edge_cluster)
            edges_cluster.append(edge_cluster)
            edge_cluster = [str(c),ca[int(len(ca)/2)]]

        if neighbor_edge:    
            print('neighbor developed')
            
        print('\n')
               
    for i,edge in enumerate(edges_cluster):
        g.edge(edge[1],edge[3],
               lhead='cluster_'+edge[2],
               ltail='cluster_'+edge[0],
               len='2',
               color = 'black',
              )

    #print(g.source)
    g.render(outfig_graphfile,view=True)
    return g



module_dict = {}
for a in axis_acronym_list_new: #NOTE test
    module_dict[a] = 'black'
    begin = axis_acronym_list_new.index('SNc_r')+2
    end = axis_acronym_list_new.index('FOTU_r')-2
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'red'
    begin = axis_acronym_list_new.index('AUDp_r')
    end = axis_acronym_list_new.index('RSPv_l')+1        
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'olive'
    begin = axis_acronym_list_new.index('MOs_l')
    end = axis_acronym_list_new.index('AD_l')+1    
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'cyan'
    begin = axis_acronym_list_new.index('COPY_r')
    end = axis_acronym_list_new.index('XII_l')+1
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'orange'
    begin = axis_acronym_list_new.index('SPIV_l')+1
    end = axis_acronym_list_new.index('SSp-un_l')
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'green'
    begin = axis_acronym_list_new.index('DT_l')+1
    end = axis_acronym_list_new.index('AHN_r')
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'purple'
    begin = axis_acronym_list_new.index('PIL_r')+1
    end = axis_acronym_list_new.index('LDT_l')  
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'pink'
    begin = axis_acronym_list_new.index('PP_r')
    end = axis_acronym_list_new.index('MD_l')+1
    if a in axis_acronym_list_new[begin:end]: module_dict[a] = 'yellow'

level = 316
hemi = False
outfig_dir = '../plots/graph'
outfig_end = 'signal'
n_clusters = 20
plot2 = 'count'
test0,test1,test2 = plot_acronym_graph(df_labels_signal_new,
                   level,hemi,plot2,
                   clusters_dict,
                   module_dict,
                   outfig_dir,outfig_end)

In [ ]:
print(axis_acronym_list_new)

# 2 get brain somata of n671 u16 region
图b上图加一下labelling bar
图b下图671*2好一些

In [ ]:
# def get_brain_somata_df(annotation,
#                     brain_path,):
#     boxin_id = []
    
#     boxin_num = 0
#     boxout_num = 0
#     tree = parse_swc(brain_path)
#     for leaf in tree:
#         i,t,x,y,z,r,p = leaf
#         try:
#             ac = annotation[round(z),round(y),round(x),]  
#             boxin_num += 1
#             boxin_id.append(ac)
#         except:
#             boxout_num += 1
#             continue
#     print(brain_path,boxin_num,boxout_num,'\n')
 
#     brain_somata_df = pd.DataFrame(np.array(np.unique(boxin_id,return_counts=True)).T,columns=['n671_u16_id','count'])    
#     return brain_somata_df
    
    
# def get_brains_somata_df(brains,
#                          annotation,
#                          somata_dir,
#                          allow0,):
#     df_brains = pd.DataFrame([])
    
#     for brain in brains:
#         brain_path = f'{somata_dir}/*/{brain}_*_stps.swc'
#         try:
#             brain_path = glob.glob(brain_path)[0]
#             print(f'\n{brain_path} appended')
#             df_brain = get_brain_somata_df(annotation = annotation,
#                                            brain_path = brain_path,)
#         except:
#             print(f'\n{brain_path} does not exists')
#             return df_brains 
        
#         if not allow0:
#             df_brain = df_brain[df_brain['n671_u16_id']!=0]
    
#         if len(df_brain):
#             df_brain['brain'] = brain
#             df_brains = pd.concat([df_brains,df_brain])
#     return df_brains   


# def get_labellingbrains_somata(labels_brains_dict,
#                                maskfile,
#                                somata_dir,
#                                allow0,):
#     annotation = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))    

#     df_labels = pd.DataFrame([])    
#     for label,brains in labels_brains_dict.items():
#         df_label = get_brains_somata_df(brains,
#                                         annotation,
#                                         somata_dir,
#                                         allow0,)    
#         if len(df_label):
#             df_label['label'] = label
#             df_labels = pd.concat([df_labels,df_label])
#     return df_labels      



# #fmost-zeng
# #n671_u16_id
# somata_dir = '../../fig1bstype/marker_regi_25'
# maskfile = f'../../assets/n671_u16.nrrd'
# df_labels_somata = get_labellingbrains_somata(labels_brains_dict = labels_brains_dict,
#                                        maskfile = maskfile,
#                                        somata_dir = somata_dir,
#                                        allow0 = allow0,)  


# df_labels_somata#acronym的顺序是数字大小的顺序

# ref

In [ ]:
# x = []
# y = []
# z = []
# a = []
# for k,v in plot_tree.items():
#     x += [v['center_u25_lr'][0]]
#     y += [v['center_u25_lr'][1]]
#     z += [v['center_u25_lr'][2]]
#     a += [k]
# center_df = pd.DataFrame(np.array([x,y,z,a]).T,columns=['x','y','z','a'])

# fig = px.scatter(x=x,y=z,)
# fig.show()
# center_df